In [2]:
import os
import shutil
import re
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from ultralytics.data.converter import convert_segment_masks_to_yolo_seg
import time
from IPython.display import display, HTML
from tqdm.notebook import tqdm

In [3]:
class JupyterLogger:
    def __init__(self):
        self.logs = []
    
    def log(self, message, level='INFO'):
        self.logs.append(f"{level}: {message}")
        display(HTML(f"<p><b>{level}</b>: {message}</p>"))
    
    def info(self, message):
        self.log(message, 'INFO')
    
    def warning(self, message):
        self.log(message, 'WARNING')
    
    def error(self, message):
        self.log(message, 'ERROR')
    
    def debug(self, message):
        self.log(message, 'DEBUG')
    
    def save_logs(self, output_dir):
        with open(os.path.join(output_dir, 'prepare_data.log'), 'w') as f:
            f.write('\n'.join(self.logs))

In [4]:
logger = JupyterLogger()

In [5]:
def preprocess_mask(mask):
    # Преобразуем бинарную маску (0 и 255) в маску с классами (0 и 1)
    return (mask > 0).astype(np.uint8)

In [6]:
def prepare_yolo_dataset(input_images_dir, input_masks_dir, output_dir, val_split=0.2):
    start_time = time.time()
    logger.info(f"Starting data preparation. Images dir: {input_images_dir}, Masks dir: {input_masks_dir}, Output dir: {output_dir}")

    os.makedirs(output_dir, exist_ok=True)
    for split in ['train', 'val']:
        os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)

    image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    mask_files = [f for f in os.listdir(input_masks_dir) if f.lower().endswith('.png')]
    
    logger.info(f"Found {len(image_files)} images and {len(mask_files)} masks")

    image_mask_pairs = {}
    for img_file in image_files:
        match = re.match(r'photo no checkersboard \((\d+)\)(_aug_(\d+))?\.jpg', img_file)
        if match:
            number = match.group(1)
            aug_num = match.group(3) or '0'  # '0' для оригинального изображения
            mask_file = f"({number}) annotated{'_aug_' + aug_num if aug_num != '0' else ''}.PNG"
            if mask_file in mask_files:
                image_mask_pairs[img_file] = mask_file
            else:
                logger.warning(f"No matching mask found for image: {img_file}")

    logger.info(f"Successfully paired {len(image_mask_pairs)} images with masks")

    image_groups = {}
    for img_file in image_mask_pairs:
        match = re.match(r'photo no checkersboard \((\d+)\)', img_file)
        if match:
            group_num = match.group(1)
            if group_num not in image_groups:
                image_groups[group_num] = []
            image_groups[group_num].append(img_file)

    train_groups, val_groups = train_test_split(list(image_groups.keys()), test_size=val_split, random_state=42)
    logger.info(f"Split data: {len(train_groups)} training groups, {len(val_groups)} validation groups")

    def process_files(groups, split):
        total = sum(len(image_groups[group]) for group in groups)
        pbar = tqdm(total=total, desc=f"Processing {split} set")
        for idx, group in enumerate(groups, 1):
            for aug_idx, img_file in enumerate(sorted(image_groups[group])):
                logger.debug(f"Processing {split} file: {img_file}")
                
                new_name = f"vein_{split}_{idx:04d}_aug_{aug_idx:02d}"
                
                src_img_path = os.path.join(input_images_dir, img_file)
                dst_img_path = os.path.join(output_dir, 'images', split, f"{new_name}.jpg")
                shutil.copy(src_img_path, dst_img_path)

                mask_file = image_mask_pairs[img_file]
                mask_path = os.path.join(input_masks_dir, mask_file)
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                
                if mask is None:
                    logger.error(f"Failed to load mask: {mask_path}")
                    continue

                processed_mask = preprocess_mask(mask)

                temp_mask_dir = os.path.join(output_dir, 'temp_masks')
                os.makedirs(temp_mask_dir, exist_ok=True)
                temp_mask_path = os.path.join(temp_mask_dir, f"{new_name}.png")
                cv2.imwrite(temp_mask_path, processed_mask)

                label_dir = os.path.join(output_dir, 'labels', split)
                os.makedirs(label_dir, exist_ok=True)

                convert_segment_masks_to_yolo_seg(temp_mask_dir, label_dir, 1)

                os.remove(temp_mask_path)
                
                pbar.update(1)
        pbar.close()

    process_files(train_groups, 'train')
    process_files(val_groups, 'val')

    shutil.rmtree(os.path.join(output_dir, 'temp_masks'))

    logger.info(f"Data preparation completed. Output saved to {output_dir}")
    logger.info(f"Training set: {len(train_groups)} groups")
    logger.info(f"Validation set: {len(val_groups)} groups")

    yaml_content = f"""
path: {output_dir}
train: images/train
val: images/val

names:
    0: vein
    """

    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    logger.info(f"Created data.yaml file at {yaml_path}")

    end_time = time.time()
    logger.info(f"Total execution time: {end_time - start_time:.2f} seconds")
    
    logger.save_logs(output_dir)

In [8]:
prepare_yolo_dataset(
    '/home/_sergei/DataspellProjects/vein-segmentation/data/26/augmented_data/images', 
    '/home/_sergei/DataspellProjects/vein-segmentation/data/26/augmented_data/masks',
    '/home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset', 
    val_split=0.1
)

Processing train set:   0%|          | 0/308 [00:00<?, ?it/s]

Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_train_0001_aug_00.png imgsz = 726 x 544
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/train/vein_train_0001_aug_00.txt imgsz = 726 x 544


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_train_0001_aug_01.png imgsz = 726 x 544
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/train/vein_train_0001_aug_01.txt imgsz = 726 x 544


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_train_0001_aug_02.png imgsz = 960 x 1280
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/train/vein_train_0001_aug_02.txt imgsz = 960 x 1280


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_train_0001_aug_03.png imgsz = 960 x 1280
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/train/vein_train_0001_aug_03.txt imgsz = 960 x 1280


...

Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_val_0004_aug_06.png imgsz = 960 x 1280
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/val/vein_val_0004_aug_06.txt imgsz = 960 x 1280


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_val_0004_aug_07.png imgsz = 960 x 1280
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/val/vein_val_0004_aug_07.txt imgsz = 960 x 1280


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_val_0004_aug_08.png imgsz = 1280 x 960
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/val/vein_val_0004_aug_08.txt imgsz = 1280 x 960


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_val_0004_aug_09.png imgsz = 960 x 1280
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/val/vein_val_0004_aug_09.txt imgsz = 960 x 1280


Processing /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/temp_masks/vein_val_0004_aug_10.png imgsz = 960 x 1280
Processed and stored at /home/_sergei/DataspellProjects/vein-segmentation/data/26/yolo_dataset/labels/val/vein_val_0004_aug_10.txt imgsz = 960 x 1280
